In [1]:
%matplotlib widget

In [2]:
import pandas as  pd 
import numpy as np
import matplotlib.pyplot as plt
import pyodbc

In [3]:
import sys
sys.path.append( 'C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\bandwidth maximization\\' )

In [4]:
from optimization_mehtods import * 
from create_event_matrices import *

In [5]:
server = 'mrwhite.intrans.iastate.edu'
database = 'sim_Andalib'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes')

In [6]:
font = {'family' : 'serif',
        'serif' : 'Times New Roman',
        'size'   : 8}
plt.rc('font', **font)

### Declare global variables

In [8]:
### input parameters
number_of_intersection = 8

cycle_length = 120

simulation_start = 300
simulation_time = 3600


In [9]:
filePath = "C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\bandwidth maximization\\"
input_parameters = pd.read_excel(filePath + "input_parameters.xlsx", sheet_name = "38th_subset")

input_parameters.loc[:, 'nextIntersection'] = input_parameters.LinkToInt
input_parameters.loc[input_parameters.sg_id == 6, 'nextIntersection'] -= 2

# input_parameters.loc[:, "offset_loc"] = 0
# input_parameters.loc[:, "offset_up"] = 0
input_parameters.LinkFromInt -= 1
input_parameters.LinkToInt -= 1

input_parameters.loc[:, "travel time"] = np.round(input_parameters.loc[:, "travel time"] , 0)
input_parameters.loc[:, "travel time"] = input_parameters.loc[:, "travel time"]%cycle_length

set_global_variables(cycle_length, number_of_intersection, simulation_start, simulation_time, input_parameters)
mat_set_global_variables(cycle_length, number_of_intersection, simulation_start, simulation_time, input_parameters)

input_parameters

,Intersection Number,intersection_id,sg_id,tt_id,LinkFromInt,LinkToInt,travel_time,saturation flow,number of lane,vpb,distance,speed,tt,split,travel time,d1,tt_next_intersection,d1_next_intersection,Prev_tt_id,nextIntersection
0,9005,1,2,30503,-1,0,5,1850,3,1.541667,0,51.45,0.0,60,0.0,190,12.9,385,0,1
1,9005,1,6,30504,1,0,5,1850,3,1.541667,662,51.45,12.9,60,0.0,210,0.0,0,30604,-1
2,9006,2,2,30603,0,1,5,1850,3,1.541667,662,51.45,12.9,60,107.0,385,12.8,585,30503,2
3,9006,2,6,30604,2,1,5,1850,3,1.541667,660,51.45,12.8,60,13.0,405,12.9,210,30704,0
4,9007,3,2,30703,1,2,5,1850,3,1.541667,660,51.45,12.8,70,94.0,585,12.8,765,30603,3
5,9007,3,6,30704,3,2,5,1850,3,1.541667,657,51.45,12.8,50,26.0,605,12.8,405,30804,1
6,9008,4,2,30803,2,3,5,1850,3,1.541667,657,51.45,12.8,44,82.0,765,13.1,970,30703,4
7,9008,4,6,30804,4,3,5,1850,3,1.541667,675,51.45,13.1,44,38.0,785,12.8,605,30904,2
8,9009,5,2,30903,3,4,5,1850,3,1.541667,675,51.45,13.1,70,68.0,970,12.8,1170,30803,5
9,9009,5,6,30904,5,4,5,1850,3,1.541667,660,51.45,12.8,70,52.0,990,13.1,785,31004,3


In [10]:
def signal_data_sql(sim_id, run_id, iteration, simulation_time, cnxn):
        signal_sql_test = "SELECT [SimTime], [SC], [SG], [state] FROM [sim_Andalib].[dbo].[signalchanges] where [StudyID]="+str(sim_id)+"and [RunID]="+str(run_id)+"and [Iteration]="+str(iteration)+ "and [SimTime]  <="+str(simulation_time)+ "Order By [SimTime]"
        signal = pd.read_sql(signal_sql_test, cnxn) 

        if(len(signal)==0): print("ERROR")

        # 1- green start, 2- yellow start, 0- Red start
        signal = signal.loc[signal.state.isin([0, 1])]
        signal["eog"] = (signal.SimTime - signal.SimTime*signal.state)
        signal["bog"] = (signal.SimTime - signal.SimTime*(1-signal.state))
        
        signal["eog"] = signal["eog"].astype(np.int64)
        signal["bog"] = signal["bog"].astype(np.int64)
        
        return signal

In [11]:
def travel_time_read_sql(sim_id, run_id, iteration, simulation_time, cnxn):
        travel_time_sql = "SELECT [Iteration],[Time],[No_],[Veh],[Trav_],[Delay_] FROM [sim_Andalib].[dbo].[traveltimes] where [StudyID]="+str(sim_id)+"and [RunID]="+str(run_id)+"and [Iteration]="+str(iteration)+"and [Time]<="+str(simulation_time)+ "Order By [Time]"
        travel_time = pd.read_sql(travel_time_sql, cnxn)
        travel_time.columns = ['Iteration','time','tt_id', 'vehNo', 'tt', 'travel_delay']
        
        return travel_time

In [12]:
def create_signal_matrices(signal_df, shifter):
        final_probability_of_green, green_time = create_probability_of_green(signal_df, shifter = 0)

        ### green prog matrix to use for probability of meeting at green (bandwidth based optimization)
        green_PrOG = np.array(final_probability_of_green)
        ### shift the matrix with travel time
        green_PrOG = align_green_with_travel_time(green_PrOG)

        final_probability_of_green, green_time = create_probability_of_green(signal_df, shifter)

        ### green prog matrix to use for probability of meeting at green (bandwidth based optimization)
        green_PrOG = np.array(final_probability_of_green)
        ### shift the matrix with travel time
        green_PrOG = align_green_with_travel_time(green_PrOG)

        eog_matrix, split_matrix, eog_cycle_num, split_cycle_num = create_eog_split_matrix(signal_df)

        ### create a green matrix with expected bog and expected eog
        final_green_time = create_green_matrix_for_bw(split_matrix, split_cycle_num, eog_matrix, eog_cycle_num)
        ### shift the matrix with travel time
        final_green_time = align_green_with_travel_time(final_green_time)

        return final_probability_of_green, final_green_time, green_PrOG


In [13]:
def create_Vehicle_matrices(final_probability_of_green, travel_time_df):
        ### create capacity matrix
        final_capacity = create_capactiy_matrix(final_probability_of_green)

        ### create vehicle distribution matrix
        final_vehicle_distribution = create_vehicle_distribution_matrix(travel_time_df)
        final_cv_distribution = create_CV_distribution_matrix(travel_time_df)
        
        return final_capacity, final_vehicle_distribution, final_cv_distribution

In [14]:
def offset_values_shift(offset_values, controller_start_number):
        offset_values = np.mod(offset_values - offset_values[0], cycle_length)
        for i in range(number_of_intersection):
                print(controller_start_number+i, " " , offset_values[i])

        return offset_values

# Put everything together

In [109]:
sim_id = 21 
run_id = 15
iteration = 1

### read signal events from sql and store to a dataframe
signal_df = signal_data_sql(sim_id, run_id, iteration, simulation_time, cnxn)

### read travel times from sql and store to a dataframe
travel_time_df = travel_time_read_sql(sim_id, run_id, iteration, simulation_time, cnxn)

final_probability_of_green, final_green_time, green_PrOG = create_signal_matrices(signal_df, 0)
final_capacity, final_vehicle_distribution, final_cv_distribution = create_Vehicle_matrices(final_probability_of_green, travel_time_df)

### Run optimization

In [98]:
set_fixed_global_matrices(final_probability_of_green, final_vehicle_distribution, final_green_time, green_PrOG, final_capacity)
set_variable_global_matrices()

In [99]:
plot_distribution(1, (32,4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
offsets = np.array([0,0,0,13,0,2,15,49], np.int16)
print_output(offsets)

System AOG: 15184.93
System POG: 15.11
System Delay: 30.71
System Bandwidth: 16
System Number of Stops: 5336.04


In [57]:
offsets= np.zeros(number_of_intersection)
method_to_use = 1
neighbors = [-40,-15,-5,-1,1,5,15,40]
number_of_iteration = 1000
number_of_random_start = 10
plotting = 0

hill_Climbing_algorithm(offsets, method_to_use, neighbors,number_of_iteration,number_of_random_start, plotting)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[3818.70833333 3611.375      3648.83333333 3583.         3911.75
 3542.375      3535.16666667 3852.08333333 3472.95833333 3583.        ]


In [22]:
print(offsets)
print_output(offsets)
offsets_2 = offset_values_shift(offsets, 9005)
print_output(offsets_2)

[ 4.  4.  4. 17.  4.  6. 19. 53.]
System AOG: 14383.47
System POG: 15.11
System Delay: 23.94
System Bandwidth: 16
System Number of Stops: 3773.67
9005   0.0
9006   0.0
9007   0.0
9008   13.0
9009   0.0
9010   2.0
9011   15.0
9012   49.0
System AOG: 14383.47
System POG: 15.11
System Delay: 23.94
System Bandwidth: 16
System Number of Stops: 3773.67


In [ ]:
# vehicles_upstream_intersection =  np.array(travel_time_df.loc[travel_time_df.tt_id == 30804, "vehNo"])
# veh_tt = travel_time_df.loc[travel_time_df.tt_id == 30704].copy() 
# print(len(vehicles_upstream_intersection), " ", len(veh_tt))

# out_df = travel_time_df.loc[(travel_time_df.tt_id == 30704) & (travel_time_df['vehNo'].isin(vehicles_upstream_intersection))] 
# print(len(out_df))

# np.savetxt("prev_tt_veh.csv", vehicles_upstream_intersection)
# veh_tt.to_excel("veh_df.xlsx")

In [54]:
def plot_distribution_method(probability_of_green, vehicle_distribution, ax_num):
        signal_seq = np.arange(0,cycle_length, 1)
        veh_seq = np.arange(0,cycle_length, 1)

        for i in range( 1,8):
                ax = axes[i-1][ax_num]
                
                ax.bar(signal_seq,probability_of_green[i][0], width = 1, color = 'green', alpha = 0.8)

                ax2=ax.twinx()
                ax2.bar(veh_seq,vehicle_distribution[i][0], width = 1, color = 'black', alpha = 0.6)
                if(i==3 or i==5): ax2.set_yticks([0, 10, 20, 30, 40, 50])
                else: ax2.set_yticks([0, 10, 20, 30, 40])
                
                if(ax_num!=4): ax2.set_yticklabels([])

In [101]:
fig, axes = plt.subplots(7, 5, sharex = True, sharey = True, figsize = (6.5, 9.5))
fig.tight_layout()
fig.subplots_adjust(wspace=0.1, hspace=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
plot_distribution_method(final_probability_of_green, final_vehicle_distribution, 4)

In [111]:
titles = ["Zero-offset", "AOG", "Delay", "PrOG", "MAXBAND"]

for i in range(len(titles)): 
        axes[0][i].set_title(titles[i], fontsize = 10)
        
for i in range(7): 
        axes[i][0].set_ylabel("Int. " + str(i+2))
        
fig.text(0.45, 0, "Cycle Second", fontsize = 10)
fig.text(-0.03, 0.45, "Green probability", rotation = 90, fontsize = 10)
fig.text(1.03, 0.45, "Vehicle frequency", rotation = 90, fontsize = 10)

directory = "C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\analysis\\"
fig.savefig(directory + "distribution2.png", dpi = 300, bbox_inches ="tight")